In [64]:
import pandas as pd
from datetime import datetime

In [47]:
%run ./util.ipynb

In [48]:
# path_startup = "/home/alex/vscode/data/original/crunchbase/startups/*.csv"
# df_startup = concat_files(path_startup, ['Organization Name'])
# df_startup.to_csv("/home/alex/vscode/data/original/crunchbase/consolidado/startups.csv")
# path_aquisicoes = "/home/alex/vscode/data/original/crunchbase/aquisicoes/*.csv"
# df_aquisicoes = concat_files(path_aquisicoes, ['Transaction Name'])
# df_aquisicoes.to_csv("/home/alex/vscode/data/original/crunchbase/consolidado/aquisicoes.csv")

In [88]:
df_startup = pd.read_csv("/home/alex/vscode/data/crunchbase/consolidado/startups.csv", index_col="index", engine='python')
df_startup['Last Funding Date'] = df_startup['Last Funding Date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d") if type(x) == str else x)
df_startup['Last_Funding_Year'] = df_startup['Last Funding Date'].apply(lambda x: x.year)
# df_startup.loc[df_startup['Last Funding Date'].str.len() == 1, 'Last Funding Date']

In [94]:
## Filter seed and pre-seed
fs_seed = df_startup['Funding Status'] == 'Seed'
lft_seed = df_startup['Last Funding Type'].isin(['Seed', 'Grant', 'Angel', 'Pre-Seed'])
old_seed = df_startup['Last_Funding_Year'] <= 2017
select_seed = fs_seed & lft_seed & old_seed
df_startup.loc[select_seed, 'Success'] = False
select_seed.sum()

321

In [95]:
## Filter series
select_series = df_startup['Last Funding Type'].isin(['Series B', 'Series C', 'Series D', 'Series E'])
df_startup.loc[select_series, 'Success'] = True
select_series.sum()

339

In [96]:
df_result = df_startup.dropna(subset=['Success'])
df_result.to_csv("/home/alex/vscode/data/processado/crunchbase/startups.csv")